In [147]:
import phonenumbers
from bs4 import BeautifulSoup
import requests
import re
from collections import namedtuple

In [ ]:
url = "http://www.canyoncreekfood.com/"

In [ ]:
content = requests.get(url)

In [ ]:
soup = BeautifulSoup(content.text, 'lxml')
body_text = soup.body.text
body_text

In [ ]:
numbers = phonenumbers.PhoneNumberMatcher(body_text, "US")
numbers

In [ ]:
numbers_to_keep = []

In [208]:
for find in numbers:
    if phonenumbers.is_possible_number(find.number) and phonenumbers.is_valid_number(find.number):
        numbers_to_keep.append(find.raw_string)
numbers_to_keep

['1(888)217-1246', '1(780)463-5511']

In [209]:
numbers_to_keep
MATCH_ALL = r'.*'


def like(string, flags=re.DOTALL):
    """
    Return a compiled regular expression that matches the given
    string with any prefix and postfix, e.g. if string = "hello",
    the returned regex matches r".*hello.*"
    """
    if isinstance(string, list):
        string_ = "|".join(map(re.escape, string))
    else:
        string_ = re.escape(string)
    if not isinstance(string_, str):
        string_ = str(string_)
    regex = MATCH_ALL + string_ + MATCH_ALL
    return re.compile(regex, flags=flags)

In [221]:
def b4_after(string, target):
    sub_str = re.escape(target)
    regex = re.compile(r"(?P<before>\S+\s)?\S*"+sub_str+"S*(?P<after>\s\S+)?", flags = re.MULTILINE | re.IGNORECASE )
    return re.finditer(regex, string)

In [233]:
keeper = namedtuple('keeper',['number', "before_afters", 'html', 'tag_text'])

In [250]:
elements = set()
for good_phone in numbers_to_keep:
        looking_for = like(good_phone)
        for elem in soup(text=looking_for):
            matches = b4_after(elem.parent.text, good_phone)
            ba = tuple(match.group() for match in matches)
            elements.add(keeper(good_phone, ba, str(elem.parent), elem.parent.text))

In [251]:
elements

{keeper(number='1(780)463-5511', before_afters=('FAX 1(780)463-5511 EMAIL',), html='<p><strong>LOCATION</strong><br/> 8704 53rd Ave, Edmonton, AB T6E5G2<br/> <strong>PHONE</strong><br/> 1(888)217-1246<br/> <strong>FAX</strong><br/> 1(780)463-5511<br/> <strong>EMAIL</strong><br/> customerfeedback@canyoncreekfood.com</p>', tag_text='LOCATION 8704 53rd Ave, Edmonton, AB T6E5G2 PHONE 1(888)217-1246 FAX 1(780)463-5511 EMAIL customerfeedback@canyoncreekfood.com'),
 keeper(number='1(888)217-1246', before_afters=('PHONE 1(888)217-1246 FAX',), html='<p><strong>LOCATION</strong><br/> 8704 53rd Ave, Edmonton, AB T6E5G2<br/> <strong>PHONE</strong><br/> 1(888)217-1246<br/> <strong>FAX</strong><br/> 1(780)463-5511<br/> <strong>EMAIL</strong><br/> customerfeedback@canyoncreekfood.com</p>', tag_text='LOCATION 8704 53rd Ave, Edmonton, AB T6E5G2 PHONE 1(888)217-1246 FAX 1(780)463-5511 EMAIL customerfeedback@canyoncreekfood.com')}

In [252]:
keeper(12341241, ["word","stuff"], "<p>", "text")

keeper(number=12341241, before_afters=['word', 'stuff'], html='<p>', tag_text='text')

In [253]:
import pandas as pd

In [254]:
number_table = pd.DataFrame(elements)

In [256]:
number_table['tag_text']

0    LOCATION 8704 53rd Ave, Edmonton, AB T6E5G2 PH...
1    LOCATION 8704 53rd Ave, Edmonton, AB T6E5G2 PH...
Name: tag_text, dtype: object